# Exercise 1
We can use the first 120 bits of IV_CFB as IV_CTR and one of the T_i for CTR will be T_0 of CFB. So we have M_0. Continue in tis way to get all M_i for CFB.

In [2]:
C1='135f75da86be0d8e74d481f793c46dc4b14dc7ec755525cb13e48d8e168c88b25ac2e6cf08d7fa11ab9fd01e2d717a1b1ec619767b31a9f0772ef4ff15506576cd86f1aceea02549610ae860b2d220834e51df8476f607bbdd9fbb3889e1e9a9fc62266e2a226ac106660e3a845987b524d50a1e7e7729788a28952509329f185080432c0492b4b929fb65df12fec6732e5be92cb8452e4b21046558c9a1a2452c98118b91ba8fc38764ea70752899750f6ad61210ef41f85d7ec1a6e390828047f82168755f57674504ed0576fcb0c0eecbb5ffe8cfb2d8f06938cba53f548ba791cf73c8b2634a712ea704a9f10e4d7df8fe3c605fb2e194f32e7540beb9737593f031686b7c251d012507d2da4e37976a3f275763672b84299d5d0c2175e1afd0bda3c125fc01c3614f3cb7831ffbf73f3bdabb571926944ded67ed2f31a26d2c8fa91e86271ebb44d2c0cdfccaa7c7afc5d913a003f3c0dfcc709729fd491d2e29269a0c557ac273bd4fb71def190502c91b7ec970d034591f3e7fd6707191328df18d4bb16dbd0c41ca94c5659232cb3e3fff3123d81d45651e'

# Exercise 2
Backtrace and Invert AES to get K. Then implemet forward implementation.

In [4]:
IV21 = "81cfb4b401265b208770983a7413c27b"
P21 = "otherwise you may just spoil eve"
C21 = "ed33655df363f972439210c6eb76bb56ac62459b100f042c115100e3e749f8ca"
IV22 = "a8de1f925258b75833f29720b121b854"
C22 = "e7b63ff33c3c97375d97b74dde53dd74d90e56fe06bd33df3fdc03fb71b4d31f1f73bfb612888e846f5ac150b51421525ebe1b35c68c5cfab6f6b17ad71dfaee8409eb1608e4f35898ac0186d9b1cda73a98f0a4d02dc32f36f79c4d42eb69a64f6e55efab19cdd9a396ee51186367f3bdad478447351c2fe147737aa703b7ceb6309b58f994d09ef7f8"

# Exercise 3
Given first i places, mapping for i+1 is fixed. IV affects only the first one. Extended key at ith place seems to depend on i-1 th position of message


In [7]:
C3 = "j:xtgqmvsaqzq.deqtf cjoy,dxwwdhjeodjxzfd.hgcgtsfxysgkhfhkdmghusexuxfsbcoherdbkfqbqoqaeho,qelokrfcjkjmehytf"

# Exercise 4
Everything is given. Just have to use elliptic curve functionality of sage.

In [11]:
p4 =  6277101735386680763835789423207666416083908700390324961279
a4 =  6277101735386680763835789423207666416083908700390324961276
b4 =  2455155546008943817740293915197451784769108058161191238065
n4 =  6277101735386680763835789423176059013767194773182842284081
Y4 =  (4880704421185356269285731254574038196082295368040177930410, 3252476952228681732222694512787255717022102608249127534923)
d4 =  4530374734388221759604160470592849519021726345429538788071
P =  (602046282375688656758213480587526111916698976636884684818, 174050332293622031404857552280219410364023488927386650641)
U4 =  (1355458087547510951659834549953094943930675781843289993571, 5254050291625999318191619702207884939318189720814143813705)
V4 =  (2990739714230682290299609801426676841596844439446139795430, 34535613716520569746498074307772112260047564877561522833)

# Exercise 5
Just draw diagram

In [12]:
C5 = "113ac90cf74391150225e9bd696a5378d1304e91b06d007cb0b5eabe7b96d40a"